# Raspagem de dados de segurança pública
### Oficina na Abraji de 2022 sobre automação com Python

1. Carregar as bibliotecas

In [1]:
import requests # importante para a raspagem de dados
from bs4 import BeautifulSoup # importante para a raspagem de dados
import pandas as pd # importante para a análise de dados e mexer com DataFrame

2. Informar o site

In [2]:
link = 'https://www.ssp.sp.gov.br/estatistica/ViolenciaMulher.aspx' # site que vamos raspar

3. Ler o HTML do site


In [3]:
page = requests.get(link) # requisitação do site que vamos raspar; o site foi informado acima
soup = BeautifulSoup(page.text) # page.text é o html e aqui ainda teremos ele identado (ou seja, organizado)

4. Informar o elemento de nosso interesse.
Por exemplo, qual é o `h3` do site?

In [4]:
# h3, citado dentro de find(), é um elemento do HTML
# ele é um texto importante, embora menos importante que h1 e h2
h3_link = soup.find('h3')
print(h3_link) #o print é usado para mostrar o que tem dentro da variável

<h3 style="color: #666;"><b>Violência Contra as Mulheres</b></h3>


5. E sem o HTML, apenas o texto?

In [5]:
# aqui vamos sobrescrever a variável
# observe que usamos 'find', e não 'find_all', então pegamos apenas o primeiro elemento
h3_link = soup.find('h3').text
print(h3_link)

Violência Contra as Mulheres


6. E se quiséssemos as `divs`?

In [6]:
# observe que agora usamos 'find_all' em vez de 'find'
# no HTML, o element 'div' é usado para guardar diversos elementos
divs_link = soup.find_all("div", id=True) # pega só as divs que têm id, por isso id é True

7. Agora podemos falar: para cada `div`, pegue a tabela.
Para isso, usamos um loop, ou seja, uma estrutura de repetição.

In [7]:
##### código já explicado acima
divs_link = soup.find_all("div", id=True) 

##### código novo
# para cada div, vamos pegar as tabelas que estão dentro (quando houver)
# aqui, pela primeira vez no notebook, vamos um loop, ou seja, uma estrutura de repetição
for div in divs_link:
    tabelas = div.find_all("table")  #table é um elemento do html para criar uma tabela
    #print(tabelas) # deixamos esta linha comentada pois há muitas linhas

8. Agora vamos usar uma estrutura condicional.
O `if` nos ajuda a começar a estruturar os dados raspados. Queremos os dados sem sujeira em um formato [tidy](https://escoladedados.org/tutoriais/tidy-data-dados-arrumados-e-5-problemas-comuns/) (ou seja, cada coluna é uma variável).

In [8]:
##### código já explicado acima
divs_link = soup.find_all("div", id=True) 

for div in divs_link:
    tabelas = div.find_all('table') 

##### código novo
    # agora usamos um if, também muito usado junto com ifelse e else, para criar condições
    if(tabelas): # se a div tiver tabela...
      titulos = div.find_all('b') # coletar o elemento 'b', onde está o texto com a data da tabela
      if(titulos): # se o elemento for encontrado... 
        data = str(titulos[0].get_text())[33:].split(' ') #  pegar a data da tabela; 33 é o número caracteres em 'Ocorrências Registradas no mês:', que vem antes da data; split separa considerando o espaço os textos que compõem a data ['junho', 'de', '2022']

9. Vamos também informar o que é cabeçalho e o que é dado na nossa `tabela`

In [26]:
##### código já explicado acima
divs_link = soup.find_all("div", id=True) 

for div in divs_link:
    tabelas = div.find_all("table")
    if(tabelas): 
      titulos = div.find_all('b')
      if(titulos): 
        data = str(titulos[0].get_text())[33:].split(' ') 

##### código novo
        final_data = [] #criamos uma lista vazia; vamos usar apenas na etapa 11
        # novamente usamos um loop para repetir uma ação
        for tabela in tabelas: # para cada tabela...
          linhas = tabela.find_all("tr") # encontre todos os elementos 'tr' - r vem de 'row', ou seja, linha
          cabecalhos = linhas[0].find_all("th") # na primeira linha encontre todos os 'th' - h vem de 'header', ou seja, cabeçalho
          #print(linhas)

10. Agora vamos fazer um novo loop para tratar os dados coletados para o cabeçalho

In [27]:
##### código já explicado acima
divs_link = soup.find_all("div", id=True) 

for div in divs_link:
    tabelas = div.find_all("table")
    if(tabelas): 
      titulos = div.find_all('b')
      if(titulos): 
        data = str(titulos[0].get_text())[33:].split(' ')
        final_data = []
        for tabela in tabelas: 
          linhas = tabela.find_all("tr") 
          cabecalhos = linhas[0].find_all("th") 
          #print(linhas)

##### código novo
          cabecalhos_all = [] # criamos uma lista vazia onde futuramente vamos colocar conteúdo
          for cabecalho in cabecalhos: # para cada item dentro de cabecalhos...
            cabecalhos_all.append(str(cabecalho.text).replace('\xa0','crime')) # substitui '\xa0' por 'crime' e guarde dentro de 'cabecalhos_all'; observe que no html a primeira coluna ta cabeçalho em branco

11. Agora vamos fazer novos loops para pegar cada número e tratá-los.



In [28]:
##### código já explicado acima
divs_link = soup.find_all("div", id=True) 

for div in divs_link:
    tabelas = div.find_all("table")
    if(tabelas): 
      titulos = div.find_all('b')
      if(titulos): 
        data = str(titulos[0].get_text())[33:].split(' ')

      final_data = [] 
      for tabela in tabelas: 
        linhas = tabela.find_all("tr") 
        cabecalhos = linhas[0].find_all("th") 
        #print(linhas)
        cabecalhos_all = []

        for cabecalho in cabecalhos: 
          cabecalhos_all.append(str(cabecalho.text).replace('\xa0','crime')) 

##### código novo

          for linha in linhas[1:]: # em cada linha, exceto na primeira (que é o cabeçalho)...
            dados = linha.find_all("td") # encontre todos os elementos 'td' - ou seja, cada célula/item da tabela 
            dados_all = [] # criamos uma lista vazia onde logo abaixo vamos colocar conteúdo

            for dado in dados: # para cada item dentro de 'dados'...
              dados_all.append(str(dado.text).replace('\n','')) # delete '\n' - quando houver - e guarde dentro o texto de 'dado' em 'dados_all'

11. Agora vamos colocar o conteúdo coletado referente a mês e ano de cada linha em 'dados_all', que já tinha os nossos dados/estatísticas de segurança. Vamos também fazer com que esses dados entrem em um arquivo final com os dados.


In [29]:
##### código já explicado acima
divs_link = soup.find_all("div", id=True) 

for div in divs_link:

    tabelas = div.find_all("table")
    if(tabelas): 
      titulos = div.find_all('b')
      if(titulos): 
        data = str(titulos[0].get_text())[33:].split(' ')

        final_data = []
        for tabela in tabelas: 
          linhas = tabela.find_all("tr") 
          cabecalhos = linhas[0].find_all("th") 
          #print(linhas)
          cabecalhos_all = []

          for cabecalho in cabecalhos: 
            cabecalhos_all.append(str(cabecalho.text).replace('\xa0','crime')) 

          for linha in linhas[1:]: 
            dados = linha.find_all("td") 
            
            dados_all = [] 
            for dado in dados: 
              dados_all.append(str(dado.text).replace('\n','')) 

##### código novo
            dados_all.append(data[0]) # mes - isso ta dentro do loop que coleta os dados; para repetir a data de cada tabela
            # nao pegamos o data[1] porque ele era 'de', que separava 'Janeiro de 2022', por exemplo
            dados_all.append(data[2]) # ano - idem

            final_data.append(dados_all) # no loop, guardamos dentro de 'final_data' o conteúdo 'dados_all'
            #print(final_df)

12. Vamos transformar o nosso dado em DataFrame, definindo também o cabeçalho

In [30]:
##### código já explicado acima
divs_link = soup.find_all("div",id=True) 

for div in divs_link:

    tabelas = div.find_all("table")
    if(tabelas): 
      titulos = div.find_all('b')
      if(titulos): 
        data = str(titulos[0].get_text())[33:].split(' ')
        
        final_data = []
        for tabela in tabelas: 
          linhas = tabela.find_all("tr") 
          cabecalhos = linhas[0].find_all("th") 
          #print(linhas)
          cabecalhos_all = []

          for cabecalho in cabecalhos: 
            cabecalhos_all.append(str(cabecalho.text).replace('\xa0','crime')) 

          for linha in linhas[1:]: 
            dados = linha.find_all("td") 
            
            dados_all = [] 
            for dado in dados:
              dados_all.append(str(dado.text).replace('\n','')) 

            dados_all.append(data[0])
            dados_all.append(data[2])
            final_data.append(dados_all) 
            #print(final_df)

##### código novo
        cabecalhos_all.append('mes') # define 'mes' como nome do cabeçalho; append = acrescentar
        cabecalhos_all.append('ano') # idem quanto a 'ano'
      final_df = pd.DataFrame(final_data, columns = cabecalhos_all) # criamos o DataFrame e definimos o cabeçalho

13. Vamos deixar todos os dados em um único objeto e exportar como arquivo de planilha CSV ou XLSX

In [36]:
##### código já explicado acima
divs_link = soup.find_all("div",id=True) 

final = pd.DataFrame()

for div in divs_link:

    tabelas = div.find_all("table")
    if(tabelas): 
      titulos = div.find_all('b')
      if(titulos): 
        data = str(titulos[0].get_text())[33:].split(' ')
        
        final_data = [] 
        for tabela in tabelas: 
          linhas = tabela.find_all("tr") 
          cabecalhos = linhas[0].find_all("th") 
          #print(linhas)
          cabecalhos_all = []

          for cabecalho in cabecalhos: 
            cabecalhos_all.append(str(cabecalho.text).replace('\xa0','crime')) 

          for linha in linhas[1:]: 
            dados = linha.find_all("td") 
            
            dados_all = [] 
            for dado in dados: 
              dados_all.append(str(dado.text).replace('\n','')) 

            dados_all.append(data[0])
            dados_all.append(data[2])
            final_data.append(dados_all) 
            #print(final_df)

        cabecalhos_all.append('mes')
        cabecalhos_all.append('ano')
      final_df = pd.DataFrame(final_data, columns = cabecalhos_all) 

##### código novo
      #print()   
      #print('3\n\n\n\n\n\n')

      final = pd.concat([final, final_df]) # para cada tabela, vamos concatenar (empilhar) e criar um DataFrame

      #print(final) 
final.to_excel("estatistica-violencia-contra-mulher.xlsx", index=False) # download para xlsx
final.to_csv("estatistica-violencia-contra-mulher.csv", index=False) # download para csv

# Obrigada!